In [ ]:
import os
import tensorflow as tf
import numpy as np
import datetime
from matplotlib import pyplot as plt
from PIL import Image
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
%matplotlib inline

# Load categories
curr_dir = os.getcwd()
# model = 'ssd_mobilenet'
# model = 'ssd_inception'
model = 'faster_rcnn'
inference_results_dir = curr_dir + '/inference_results_'+ model
try:
    os.mkdir(inference_results_dir)
except OSError:
    pass
frozen_graph_file = curr_dir + '/inference_graphs/'+model+'/frozen_inference_graph.pb'
PATH_TO_LABELS = curr_dir + '/label_map.pbtxt'
NUM_CLASSES =4
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)



In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()

    with tf.gfile.GFile(frozen_graph_file, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    sess = tf.Session(graph=detection_graph, config=config)

In [ ]:
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [ ]:
    def get_classification(self, image):
        """Determines the color of the traffic light in the image

        Args:
            image (cv::Mat): image containing the traffic light

        Returns:
            int: ID of traffic light color (specified in styx_msgs/TrafficLight)

        """
        traffic_light = TrafficLight.UNKNOWN

        image_expanded = np.expand_dims(image, axis=0)
        with self.detection_graph.as_default():
            (boxes, scores, classes, num) = self.sess.run(
                [self.detection_boxes, self.detection_scores,
                 self.detection_classes, self.num_detections],
                feed_dict={self.image_tensor: image_expanded})

        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes).astype(np.int32)

        min_score_threshold = .50
        
        class_name = None
        for i in range(boxes.shape[0]):
            if scores is None or scores[i] > min_score_threshold:

                class_name = self.category_index[classes[i]]['name']

                if class_name == 'Red':
                    traffic_light = TrafficLight.RED
                elif class_name == 'Green':
                    traffic_light = TrafficLight.GREEN
                elif class_name == 'Yellow':
                    traffic_light = TrafficLight.YELLOW

                self.image_np_deep = image

        # rospy.loginfo('tl found: {}'.format(class_name))

        return traffic_light

In [ ]:
inference_images = []
for dirname, dirnames, filenames in os.walk('inference_images'):
    inference_images = filenames

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
IMAGE_SIZE = (10, 6)
durations =[]
i=0
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    for filename in inference_images:
        image_path = os.path.join('inference_images', filename)
        image = Image.open(image_path)
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        a = datetime.datetime.now()
        (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
        b = datetime.datetime.now()
        millis = (b - a).total_seconds() * 1000
        i = i + 1
        # ignore the first inference as it's usually an outlier(very slow)
        if i > 1:
            durations.append(millis)
        print('Inference took: {}'.format(millis))
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)
        fig = plt.figure(figsize=IMAGE_SIZE)
        plt.imshow(image_np)
        plt.savefig(inference_results_dir + '/'+ filename)
#         plt.close()
print('Average inference took: {}'.format(sum(durations)/len(durations)))
        